<a href="https://colab.research.google.com/github/LC1332/Zero-Haruhi/blob/main/notebook/%E5%AF%B9%E8%AF%9D%E6%8A%BD%E5%8F%96_%E4%BD%BF%E7%94%A8colab%E8%BF%9B%E8%A1%8Cfinetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade huggingface_hub
# !huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credent

In [2]:
!git clone https://github.com/hiyouga/LLaMA-Factory.git

Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 6750, done.
remote: Counting objects: 100% (769/769), done.
remote: Compressing objects: 100% (316/316), done.
remote: Total 6750 (delta 506), reused 661 (delta 451), pack-reused 5981
Receiving objects: 100% (6750/6750), 204.48 MiB | 20.83 MiB/s, done.
Resolving deltas: 100% (4857/4857), done.
Updating files: 100% (140/140), done.


In [3]:
%cd LLaMA-Factory
!pip install -q -r requirements.txt
!pip install -q einops

/content/LLaMA-Factory
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.7 MB/s eta 0:00:00

In [4]:
!mkdir /content/dataset

In [5]:
!ls /content/LLaMA-Factory/dataset

ls: cannot access '/content/LLaMA-Factory/dataset': No such file or directory


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!ls /content/drive/MyDrive/CardBuild/HaruhiZero/*dialogue*

/content/drive/MyDrive/CardBuild/HaruhiZero/chinese_dialogue_clean.jsonl
/content/drive/MyDrive/CardBuild/HaruhiZero/dialogue_extract.jsonl
/content/drive/MyDrive/CardBuild/HaruhiZero/dialogue_negative.jsonl
/content/drive/MyDrive/CardBuild/HaruhiZero/english_dialogue_clean.jsonl
/content/drive/MyDrive/CardBuild/HaruhiZero/english_dialogue_extract.jsonl



请为我实现一段python代码，把input_path下的几个file_names对应的jsonl文件(utf-8, ensure_ascii=False) 进行读取

然后以json格式写入target_file

In [48]:
input_path = "/content/drive/MyDrive/CardBuild/HaruhiZero/"
file_names = ["chinese_dialogue_clean.jsonl","dialogue_negative.jsonl","english_dialogue_clean.jsonl"]
# file_names = ["chinese_dialogue_clean.jsonl"]
target_path = "/content/LLaMA-Factory/data/"

import json
import os

target_file = "/content/LLaMA-Factory/data/dialogue_merged.json"

datas = []
for file_name in file_names:
    with open(os.path.join(input_path, file_name), 'r', encoding='utf-8') as f:
        for line in f:
            data = json.loads(line)
            # delete the key except system and conversations
            if "source" in data:
                del data["source"]
            datas.append(data)

with open(target_file, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

target_file_jsonl = "/content/LLaMA-Factory/data/dialogue_merged.jsonl"


with open(target_file_jsonl, 'w', encoding='utf-8') as f:
    for data in datas:
        f.write(json.dumps(data, ensure_ascii=False) + '\n')


In [49]:
for data in datas:
    if "system" not in data:
        print("system not in data")
    elif "conversations" not in data:
        print("conversations not in data")
    elif len(data["conversations"]) != 2:
        print("conversations is empty")
    else:
        text = data["conversations"][0]
        continue
    break

请为我实现一段python代码，对于/content/LLaMA-Factory/data/dataset_info.json 备份到dataset_info_back.json

In [14]:
import shutil
import os

original_file = '/content/LLaMA-Factory/data/dataset_info.json'
backup_file = '/content/LLaMA-Factory/data/dataset_info_back.json'

# Make a copy of the original file
shutil.copy(original_file, backup_file)

# Verify copy succeeded
if os.path.exists(backup_file):
    print(f'Backup of {original_file} to {backup_file} succeeded!')
else:
    print(f'Backup of {original_file} to {backup_file} failed!')

Backup of /content/LLaMA-Factory/data/dataset_info.json to /content/LLaMA-Factory/data/dataset_info_back.json succeeded!


为我实现一段python代码，复制/content/LLaMA-Factory/data/dataset_info.json中的信息

写入到 /content/LLaMA-Factory/data/dataset_info.json

并且在末尾添加


```json
"haruhi_dialogue_extract": {
  "file_name": "dialogue_merged.json",
  "columns": {
    "messages": "conversations",
    "system": "system",
    "tools": "tools"
  },
  "tags": {
    "role_tag": "from",
    "content_tag": "value"
  }
}
```

In [50]:
import json

original_file = '/content/LLaMA-Factory/data/dataset_info.json'
new_file = '/content/LLaMA-Factory/data/dataset_info.json'

with open(original_file, 'r') as f:
  data = json.load(f)

data['haruhi_dialogue_extract'] = {
  "file_name": "/content/LLaMA-Factory/data/dialogue_merged.jsonl",
  "formatting":"sharegpt",
  "columns": {
    "messages": "conversations",
    "system": "system",
  },
  "tags": {
    "role_tag": "from",
    "content_tag": "value",
    "user_tag": "human",
    "assistant_tag":"gpt",
  }
}

with open(new_file, 'w') as f:
  json.dump(data, f, indent=2)

print('Data copied and new info appended to', new_file)

Data copied and new info appended to /content/LLaMA-Factory/data/dataset_info.json


In [19]:
!pip install -q tiktoken transformers_stream_generator

  Preparing metadata (setup.py) ... done


In [ ]:
!pip install -q datasets

In [ ]:
! CUDA_VISIBLE_DEVICES=0 python src/train_bash.py \
    --stage sft \
    --model_name_or_path  Qwen/Qwen-1_8B-Chat\
    --do_train True\
    --dataset haruhi_dialogue_extract \
    --template qwen \
    --finetuning_type lora \
    --lora_target c_attn \
    --lora_rank 16 \
    --lora_dropout 0.05 \
    --output_dir qwen_1_8-finetuned \
    --overwrite_cache \
    --per_device_train_batch_size 8 \
    --gradient_accumulation_steps 1 \
    --lr_scheduler_type cosine \
    --logging_steps 100 \
    --save_steps 1000 \
    --learning_rate 2e-4 \
    --num_train_epochs 3.0 \
    --plot_loss \
    --fp16 True

2024-01-25 01:43:49.540722: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-25 01:43:49.540776: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-25 01:43:49.542653: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-25 01:43:50.884157: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
01/25/2024 01:43:53 - WARNING - llmtuner.hparams.parser - `ddp_find_unused_parameters` needs to be set as False for LoRA in DDP training.
[INFO|training_args.py:1828] 2024-01-25 01:43:53,121 >> PyTorch: setting up devices
/usr/local/lib/pyth